# Welcome!

Jupyter repo for this tutorial: https://github.com/reuven/2024-pycon-decorators/

# Agenda

1. What are decorators?
2. Writing your first decorator
3. Outer function storage
4. Inputs and outputs
5. Decorators that take arguments
6. Nested decorators
7. Decorating classes
8. Writing decorators as classes